In [15]:
import tensorflow as tf
from tensorflow.keras import Model
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.6.0


In [8]:
# Load data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to floating point numbers
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# tf.data.Datasets
Datasets creates input pipeline from raw data like csv's to model-ready input, and can perform transformations like:

- Shuffling: helps reduce variance, making sure the models remain general and don't overfit
- Batching: by calculating SGD on a single batch you get a good estimate of the true gradient, saving computation time by not having to calculate the true gradient over the entire dataset every single time

In [14]:
# Shuffle + Batch
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [16]:
class MyModel(Model):
    '''
    An ANN model. Inherits from tf.Model
    '''
    def __init__(self):
        '''
        Define the layers that will be used
        '''
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation='relu')
        self.d2 = tf.keras.layers.Dense(10)
    
    def call(self, x):
        '''
        Model's forward pass. Can add optional training
        argument (bool) to specify different behavior
        in training versus inference (test)
        '''
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
# Create an instance of the model
model = MyModel()

# Choose optimizer and loss fn for training
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Select metrics to measure the loss and accuracy of model
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# TF Methods
- tf.function() compiles a function into a callable TensorFlow graph
- tf.GradientTape() records operations for automatic differentiation onto a tape. Then use that tape to compute the gradients of a computation

In [20]:
# Use tf.GradientTape to train model
@tf.function # turns fn into tf graph
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True only if layers with different
        # behavior during training vs inference (e.g. Dropout)
        predictions = model(images, training=True) # x
        loss = loss_object(labels, predictions) # y = x^2
    # tape.gradients(y, x) is deriv of y wrt x
    gradients = tape.gradient(loss, model.trainable_variables) # dloss / dmodel
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [22]:
# Test the model
@tf.function
def test_step(images, labels):
    # training=False only if layers with different
    # behavior during training versus inference/test (e.g. Dropout)
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [23]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

2021-12-27 13:37:12.694541: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1, Loss: 0.13220779597759247, Accuracy: 96.0, Test Loss: 0.06056513264775276, Test Accuracy: 98.0999984741211
Epoch 2, Loss: 0.04262334853410721, Accuracy: 98.69999694824219, Test Loss: 0.04791032895445824, Test Accuracy: 98.44999694824219
Epoch 3, Loss: 0.021768298000097275, Accuracy: 99.2683334350586, Test Loss: 0.047774624079465866, Test Accuracy: 98.56999969482422
Epoch 4, Loss: 0.014486235566437244, Accuracy: 99.5433349609375, Test Loss: 0.0531855933368206, Test Accuracy: 98.3699951171875
Epoch 5, Loss: 0.008839472196996212, Accuracy: 99.69999694824219, Test Loss: 0.0542929470539093, Test Accuracy: 98.54999542236328
